<a href="https://colab.research.google.com/github/atherfawaz/BERT-Supervised/blob/master/RoBERTa_Multi_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Supervised Learning with BERT

***Question*: Supervised Learning with BERT
The Astrological department believes that a person's astrological sign can be guessed from their behavior. An organization is collecting blog-posts of different people from various sources. You have been tasked to build a Deep Learning model that can use these posts data of individuals to predict which star group out of 12 does an individual belong to. You also need to tell the gender of that person.**

# Dataset
The file name would contain the gender, age, occupation, and astrological sign of the blooger. For example, 4115891.male.24.Student.Leo.xml is one file. A single file will contain a set of blogs separated by date. To illustrate, this is what a sample file looks like:

```
<Blog>
  <date>31,May,2004</date>
    <post>
      Well, everyone got up and going this morning.  It's still raining, but that's okay with me.  Sort of suits my mood.  I could easily have stayed home in bed with my book and the cats.  This has been a lot of rain though!..
    </post>
</Blog
```

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/My Drive/Ebryx/blogs_train

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/Ebryx/blogs_train


In [2]:
!pip install -q transformers
!curl -q https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --apt-packages libomp5 libopenblas-dev

     |████████████████████████████████| 778kB 3.4MB/s 
     |████████████████████████████████| 890kB 24.4MB/s 
     |████████████████████████████████| 1.1MB 34.5MB/s 
     |████████████████████████████████| 3.0MB 47.5MB/s 
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5115  100  5115    0     0  18010      0 --:--:-- --:--:-- --:--:-- 17947
Updating... This may take around 2 minutes.
Updating TPU runtime to pytorch-dev20200515 ...
Uninstalling torch-1.6.0+cu101:
     |████████████████████████████████| 61kB 2.7MB/s 
  Found existing installation: google-api-python-client 1.7.12
    Uninstalling google-api-python-client-1.7.12:
      Successfully uninstalled google-api-python-client-1.7.12
Done updating TPU runtime
  Successfully uninstalled torch-1.6.0+cu101
Uninstalling torchvision-0.7.0+cu101:
  Successfully uninstalled torchvision-0.7.0+cu101
Copying gs://tpu-pytorch/w

In [17]:
import numpy as np
import pandas as pd
import torch
from sklearn import metrics
import transformers
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import RobertaConfig, RobertaModel, RobertaTokenizerFast, RobertaForSequenceClassification
from transformers import BertTokenizer, BertModel, BertConfig

In [ ]:
# TPU
import torch_xla
import torch_xla.core.xla_model as xm
from sklearn.utils import shuffle
device = xm.xla_device()

# Hyperparameters

In [5]:
MAX_LEN = 256
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 8
EPOCHS = 1
LEARNING_RATE = 1e-05
#tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_size = 0.8

# Parsing the dataset
Parsing the dataset from separate files into a Pandas Dataframe for displaying and easy access. Some XML files contain encoding issues and the problematic contents of those files have been replaced by random number. While this could impact the accuracy of the model later, the effect would not be that big.

In [9]:
import re

%cd /content/gdrive/My Drive/Ebryx/blogs_train
df = pd.read_csv('/content/gdrive/My Drive/Ebryx/dataset.csv')
df = df[['Post', 'Sign', 'Gender']]

encode_dict = {}
m_arr = np.identity(12, dtype=int)

def encode_cat(x):
  if x not in encode_dict.keys():
    encode_dict[x]=len(encode_dict)
  return encode_dict[x]

def take_alphabets(x):
  x = str(x).lower()
  re.sub(r'\W+', '', x)
  re.sub('[^A-Za-z]+', '', x)
  return x

def one_hot(x):
  return m_arr[x]

df['Sign'] = df['Sign'].apply(lambda x: encode_cat(x))
df['Post'] = df['Post'].str.replace('\r','').str.replace('\t','').str.replace('\xa0', '').str.replace('\n', '').str.replace('urlLink', '')
df['Post'] = df['Post'].apply(lambda x: take_alphabets(x))
df['Sign'] = df['Sign'].apply(lambda x: one_hot(x))

df = df [['Post', 'Sign']]

df = shuffle(df)
df.head(10)

/content/gdrive/My Drive/Ebryx/blogs_train


,Post,Sign
45461,had a gethering with my buddies.....,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]"
269070,well considering that i am newly single...,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]"
358879,life is all about ass...either you...,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]"
2728,in my past lives i have been ...,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]"
201278,"ok so my mom randomly says yesterday ""h...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]"
283994,glenn mrs solastri is droping...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
81848,if you ever want to crash a system ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]"
272644,"oh, an interesting thing, i think. my ...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]"
360597,sum artists i hate...or bands.... ...,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]"
198981,i caught it the first time. i did ...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


# Cleaning the dataset with NLTK

In [ ]:
import re
import nltk
import string
from progressbar import ProgressBar
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

pbar = ProgressBar()
nltk.download('stopwords');
nltk.download('punkt')
stops = set(stopwords.words("english"))  

class preprocess():
  def __init__(self, dataframe):
    self.df = dataframe

  def identify_tokens(self, row):
    text = row['Post']
    tokens = nltk.word_tokenize(str(text))
    # taken only words (not punctuation)
    token_words = [w for w in tokens if w.isalpha()]
    return token_words
  
  def stem_list(self, row):
    stemming = PorterStemmer()
    my_list = row['words']
    stemmed_list = [stemming.stem(word) for word in my_list]
    return (stemmed_list)

  def remove_stops(self, row):
    my_list = row['stemmed_words']
    meaningful_words = [w for w in my_list if not w in stops]
    return (meaningful_words)
  
  def rejoin_words(self, row):
    my_list = row['stem_meaningful']
    joined_words = ( " ".join(my_list))
    return joined_words

  def preprocess_dataframe(self):
    print("Converting letters to lowercase...")
    self.df['Post'].str.lower()
    print("Tokenizing sentences in dataset...")
    self.df['words'] = self.df.apply(self.identify_tokens, axis=1)
    print("Stemming words in dataset...")
    self.df['stemmed_words'] = self.df.apply(self.stem_list, axis=1)
    print("Removing stop words in dataset...")
    self.df['stem_meaningful'] = self.df.apply(self.remove_stops, axis=1)
    self.df['Post'] = self.df.apply(self.rejoin_words, axis=1)
    print("Pre-processing complete.")
    
    return self.df

df = preprocess(df).preprocess_dataframe()
df = df[['Post', 'Sign']]

#df.to_csv('/tmp/training_processed.csv', index=False)


df.head(10)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Converting letters to lowercase...
Tokenizing sentences in dataset...
Stemming words in dataset...
Removing stop words in dataset...
Pre-processing complete.


,Post,Sign
26014,whi I piss I went bed last night onli wake eno...,3
363023,player play like hack anoth poor perform playe...,7
16331,burn mail list forward thi Yo I onli wa friday...,6
184430,happi birthday nutter butter,0
52267,woah back frm japan much fun wan na tok abt ti...,3
16634,AP report target new jersey soon attent wa su ...,7
191292,I play around blogger templat result see anyon...,9
171003,I love quizilla fraggl rock charact brought qu...,9
19610,I tire want feel faithless lost surfac I know ...,0
2451,quick post befor work jami call around noon to...,8


In [ ]:
df.to_csv('/content/gdrive/My Drive/Ebryx/processed_dataset.csv', encoding='utf-8', index=False)
df.head(10)

,Post,Sign
26014,whi I piss I went bed last night onli wake eno...,3
363023,player play like hack anoth poor perform playe...,7
16331,burn mail list forward thi Yo I onli wa friday...,6
184430,happi birthday nutter butter,0
52267,woah back frm japan much fun wan na tok abt ti...,3
16634,AP report target new jersey soon attent wa su ...,7
191292,I play around blogger templat result see anyon...,9
171003,I love quizilla fraggl rock charact brought qu...,9
19610,I tire want feel faithless lost surfac I know ...,0
2451,quick post befor work jami call around noon to...,8


In [ ]:
df = pd.read_csv('/content/gdrive/My Drive/Ebryx/processed_dataset.csv')
df.head()

,Post,Sign
0,whi I piss I went bed last night onli wake eno...,3
1,player play like hack anoth poor perform playe...,7
2,burn mail list forward thi Yo I onli wa friday...,6
3,happi birthday nutter butter,0
4,woah back frm japan much fun wan na tok abt ti...,3


# Creating a class for encapsulating the dataset

In [14]:
class CustomDataset(Dataset):
  
  def __init__(self, dataframe, tokenizer, max_len):
      self.tokenizer = tokenizer
      self.data = dataframe
      self.post = dataframe.Post
      self.targets = dataframe.Sign
      self.max_len = max_len

  def __len__(self):
      return len(self.post)

  def __getitem__(self, index):
      post = str(self.post[index])
      post = " ".join(post.split())

      inputs = self.tokenizer.encode_plus(
          post,
          None,
          add_special_tokens=True,
          max_length=self.max_len,
          truncation=True,
          pad_to_max_length=True,
          return_token_type_ids=True
      )
      ids = inputs['input_ids']
      mask = inputs['attention_mask']
      token_type_ids = inputs["token_type_ids"]

      return {
          'ids': torch.tensor(ids, dtype=torch.long),
          'mask': torch.tensor(mask, dtype=torch.long),
          'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
          'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

# Separating into training and validation sets

In [15]:
# Creating the dataset and dataloader for the neural network

train_size = 0.8
train_dataset = df.sample(frac=train_size,random_state=200)
test_dataset = df.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)


print("FULL Dataset: {}".format(df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = CustomDataset(train_dataset, tokenizer, MAX_LEN)
testing_set = CustomDataset(test_dataset, tokenizer, MAX_LEN)

train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

FULL Dataset: (380720, 2)
TRAIN Dataset: (304576, 2)
TEST Dataset: (76144, 2)


In [20]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 

class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l1 = transformers.BertModel.from_pretrained('bert-base-uncased')
        self.l2 = torch.nn.Dropout(0.3)
        self.l3 = torch.nn.Linear(768, 12)
    
    def forward(self, ids, mask, token_type_ids):
        _, output_1= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids)
        output_2 = self.l2(output_1)
        output = self.l3(output_2)
        return output

model = BERTClass()
model.to(device)

def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
def train(epoch):
  model.train()
  for i,data in enumerate(training_loader, 0):
    ids = data['ids'].to(device, dtype = torch.long)
    mask = data['mask'].to(device, dtype = torch.long)
    token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
    targets = data['targets'].to(device, dtype = torch.float)

    outputs = model(ids, mask, token_type_ids)

    optimizer.zero_grad()
    loss = loss_fn(outputs, targets)
    if i%100==0:
        print(f'[ {i} ] Loss: {loss.item()}')
    
    optimizer.zero_grad()
    loss.backward()
    #optimizer.step()
    xm.optimizer_step(optimizer)
    xm.mark_step()

for epoch in range(EPOCHS):
  train(epoch)

[ 0 ] Loss: 0.7125088572502136
[ 100 ] Loss: 0.31261903047561646
[ 200 ] Loss: 0.29223331809043884
[ 300 ] Loss: 0.28564220666885376
[ 400 ] Loss: 0.2900067865848541
[ 500 ] Loss: 0.27668482065200806
[ 600 ] Loss: 0.2913244366645813
[ 700 ] Loss: 0.28977906703948975
[ 800 ] Loss: 0.2860506474971771
[ 900 ] Loss: 0.2965611219406128



# Epoch 1

In [ ]:

model = ModelClass()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)
model.to(device)
loss_function = torch.nn.CrossEntropyLoss()

for epoch in range(EPOCHS):
  steps = 0
  n_tr_correct, n_val_correct = 0, 0
  tr_loss, val_loss = 0, 0
  nb_tr_examples, nb_val_examples = 0, 0
  for (i, data), (j, val_data) in zip(enumerate(training_loader, 0), enumerate(validation_loader, 0)):
      model.train()
      ids = data['ids'].to(device, dtype = torch.long)
      mask = data['mask'].to(device, dtype = torch.long)
      targets = data['targets'].to(device, dtype = torch.long)

      outputs = model(ids, mask).squeeze()
      optimizer.zero_grad()
      loss = loss_function(outputs, targets)
      big_val, big_idx = torch.max(outputs.data, dim=1)
      
      n_tr_correct += calcuate_accu(big_idx, targets)
      nb_tr_examples += targets.size(0)

      tr_loss += loss.item()
      
      if i%100==0:
        loss_step = tr_loss/100
        train_acc = n_tr_correct/nb_tr_examples
        tr_loss, n_tr_correct, nb_tr_examples = 0, 0, 0
        print(f"[{i}] train_loss: {loss_step:.3f} train_acc: {train_acc:.3f}", end="    ") 
      
      optimizer.zero_grad()
      loss.backward()
      
      # # When using GPU or GPU
      # optimizer.step()
      
      # When using TPU
      xm.optimizer_step(optimizer)
      xm.mark_step()

      model.eval()
      with torch.no_grad():
          ids = val_data['ids'].to(device, dtype = torch.long)
          mask = val_data['mask'].to(device, dtype = torch.long)
          targets = val_data['targets'].to(device, dtype = torch.long)

          outputs = model(ids, mask).squeeze()
          loss = loss_function(outputs, targets)
          big_val, big_idx = torch.max(outputs.data, dim=1)
          
          n_val_correct += calcuate_accu(big_idx, targets)
          nb_val_examples += targets.size(0)

          val_loss += loss.item()
          
          if j%100==0:
            val_step = val_loss/100
            val_acc = n_val_correct/nb_val_examples
            val_loss, n_val_correct, nb_val_examples = 0, 0, 0
            print(f"val_loss: {val_step:.3f} val_acc : {val_acc:.3f}") 

# Saving model

In [ ]:
PATH = '/content/gdrive/My Drive/Ebryx/roberta_12class.pt'
torch.save(model.state_dict(), PATH)

# Loading trained model

In [ ]:
PATH = '/content/gdrive/My Drive/Ebryx/roberta_12class.pt'
model = ModelClass()
model.load_state_dict(torch.load(PATH))
#model.eval()
#print(model)

<All keys matched successfully>

# Epoch 2-N

In [ ]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)
model.to(device)
loss_function = torch.nn.CrossEntropyLoss()


steps = 0
n_tr_correct, n_val_correct = 0, 0
tr_loss, val_loss = 0, 0
nb_tr_examples, nb_val_examples = 0, 0
for (i, data), (j, val_data) in zip(enumerate(training_loader, 0), enumerate(validation_loader, 0)):
    model.train()
    ids = data['ids'].to(device, dtype = torch.long)
    mask = data['mask'].to(device, dtype = torch.long)
    targets = data['targets'].to(device, dtype = torch.long)

    outputs = model(ids, mask).squeeze()
    optimizer.zero_grad()
    loss = loss_function(outputs, targets)
    big_val, big_idx = torch.max(outputs.data, dim=1)
    
    n_tr_correct += calcuate_accu(big_idx, targets)
    nb_tr_examples += targets.size(0)

    tr_loss += loss.item()
    
    if i%100==0:
      loss_step = tr_loss/100
      train_acc = n_tr_correct/nb_tr_examples
      tr_loss, n_tr_correct, nb_tr_examples = 0, 0, 0
      print(f"[{i}] train_loss: {loss_step:.3f} train_acc: {train_acc:.3f}", end="    ") 
    
    optimizer.zero_grad()
    loss.backward()
    
    # # When using GPU or GPU
    # optimizer.step()
    
    # When using TPU
    xm.optimizer_step(optimizer)
    xm.mark_step()

    model.eval()
    with torch.no_grad():
        ids = val_data['ids'].to(device, dtype = torch.long)
        mask = val_data['mask'].to(device, dtype = torch.long)
        targets = val_data['targets'].to(device, dtype = torch.long)

        outputs = model(ids, mask).squeeze()
        loss = loss_function(outputs, targets)
        big_val, big_idx = torch.max(outputs.data, dim=1)
        
        n_val_correct += calcuate_accu(big_idx, targets)
        nb_val_examples += targets.size(0)

        val_loss += loss.item()
        
        if j%100==0:
          val_step = val_loss/100
          val_acc = n_val_correct/nb_val_examples
          val_loss, n_val_correct, nb_val_examples = 0, 0, 0
          print(f"val_loss: {val_step:.3f} val_acc : {val_acc:.3f}") 

[0] train_loss: 0.018 train_acc: 0.375    val_loss: 0.020 val_acc : 0.250
[100] train_loss: 2.022 train_acc: 0.308    val_loss: 2.012 val_acc : 0.306
[200] train_loss: 2.034 train_acc: 0.322    val_loss: 2.008 val_acc : 0.310
[300] train_loss: 2.006 train_acc: 0.323    val_loss: 1.973 val_acc : 0.334
[400] train_loss: 2.005 train_acc: 0.319    val_loss: 1.967 val_acc : 0.323
[500] train_loss: 2.032 train_acc: 0.309    val_loss: 2.011 val_acc : 0.331
[600] train_loss: 2.006 train_acc: 0.319    val_loss: 1.906 val_acc : 0.371
[700] train_loss: 2.035 train_acc: 0.304    val_loss: 2.020 val_acc : 0.335
[800] train_loss: 1.996 train_acc: 0.321    val_loss: 1.947 val_acc : 0.346
[900] train_loss: 2.030 train_acc: 0.318    val_loss: 1.965 val_acc : 0.344
[1000] train_loss: 2.039 train_acc: 0.314    val_loss: 1.976 val_acc : 0.321
[1100] train_loss: 2.009 train_acc: 0.309    val_loss: 1.942 val_acc : 0.340
[1200] train_loss: 2.023 train_acc: 0.315    val_loss: 1.932 val_acc : 0.344
[1300] trai

KeyboardInterrupt: ignored

In [ ]:
PATH = '/content/gdrive/My Drive/Ebryx/roberta_12class.pt'
torch.save(model.state_dict(), PATH)